In [72]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pickle
import pandas as pd
import time
import json

In [64]:
# Function to scrape recipes.
def get_links(item):
    chromedriver = "/Applications/chromedriver"
    driver = webdriver.Chrome(chromedriver)
    url = 'https://www.thekitchn.com/search?filter=recipes&q={}/'
    url = url.format(item)
    kitchn_links = []
    driver.get(url)
    for r in range(39):
        time.sleep(4)
        page = driver.page_source
        soup = BeautifulSoup(page, 'lxml') 
        body = soup.find_all('a', {'data-gtm-teaser-slot': True})
        for b in body:
            kitchn_links.append(b.get('href'))
        try:
            next_page = driver.find_element_by_class_name("SearchPagination__next-link")
            next_page.send_keys(Keys.RETURN)
        except:
            driver.close()
    kitchn_links = list(set(kitchn_links))
    kitchn_links.remove('/recipe-take-these-3-upgraded-chicken-sandwiches-back-to-school-247393')
    kitchn_links.remove('/spicehunter-recipe-spaghetti-squash-jambalaya-264607')
    kitchn_links.remove('/in-eric-gowers-kitchen-the-bar-54356')
    kitchn_links.remove('/vijay-jw-block-test-11838515')
    kitchn_links.remove('/recipe-zuppa-toscana-234423')
    return kitchn_links

In [65]:
kitchn_links = get_links('chicken')

In [73]:
# Function to scrape and create recipe database
def get_recipes(links):
    chromedriver = "/Applications/chromedriver"
    driver = webdriver.Chrome(chromedriver)
    recipe_d = {}
    idx = 0
    for k in links:
        url = 'https://www.thekitchn.com{}'
        new_url = url.format(k)
        driver.get(new_url)
        page = driver.page_source 
        soup = BeautifulSoup(page, 'lxml')  
        title = []
        title.append(soup.find_all('h1', {'class': True})[0].text)
        tags = []
        soup_tags = soup.find_all('li', {'class': 'NutritionalGuide__label'})
        for s in soup_tags:
            tags.append(s.text.replace('-', ' '))
        soup_tags = json.loads(soup.find_all('script', {'id': 'liftigniter-metadata'})[0].text)['tag']
        for s in soup_tags:
            tags.append(s)
        ingredients = []
        soup_ingred = soup.find_all('li', {'class': 'Recipe__ingredient'})
        for i in soup_ingred:
            temp = []
            for j in i:
                temp.append(j.text.strip())
            ingredients.append(' '.join(temp))
        soup_stars = soup.find_all('lineargradient', {'id': True})
        if len(soup_stars) > 0:
            stars = soup_stars[0].get('id')[-3:]
            if stars.count('-') > 1: 
                stars = float(stars[-1])
            else:
                stars = float(stars.replace('-', '.'))
        else:
            weird_stars = soup.find('li', {'class': 'jsx-1782090057 RecipeRating__star RecipeRating__star--5'})
            if 'RecipeRating__star-shape--full' in str(weird_stars):
                stars = 5.0       
        reviews = soup.find_all('span', {'class': 'jsx-1985711476 RecipeRating__number'})[0].text
        reviews = int(reviews[:-7].strip())
        instructions = []
        soup_instruct = soup.find_all('ol', {'class': 'Recipe__instructions'})
        for s in soup_instruct:
            instructions.append(s.text)
        instructions = ''.join(instructions).strip()
        try:
            servings = soup.find('span', {'class': 'Recipe__yield-entry'}).contents[1].text
        except:
            pass
        nutrient = []
        for s in soup.find_all('span', {'class': 'NutritionalGuide__nutrient-name'}):
            nutrient.append(s.text.strip())
        quantity = []
        for s in soup.find_all('span', {'class': 'NutritionalGuide__nutrient-quantity'}):
            quantity.append(s.text)
        d = {}
        d['nutrition'] = dict(zip(nutrient, quantity))

        try:
            image = soup.find_all('img', {'src', True})[0].get('srcset').split(' ')[0]
        except:
            image = soup.find_all('img', {'src', True})[1].get('srcset').split(' ')[0]

        d['title'] = title
        d['instructions'] = instructions
        d['servings'] = servings
        d['ingredient'] = ingredients
        d['tags'] = tags
        d['reviews'] = reviews
        d['stars'] = stars
        d['link'] = 'https://www.thekitchn.com' + k
        d['image'] = image
        recipe_d[idx] = d 
        idx += 1

    recipe_db = pd.DataFrame.from_dict(recipe_d, orient = 'index')  
    driver.close()
    return recipe_db

In [ ]:
recipe_db = get_recipes(kitchn_links)

In [85]:
# Function to clean database and drop recipes that contain other proteins.
def drop_proteins(database, *args):
    recipe_db = database
    recipe_db['tags'] = recipe_db['tags'].map(lambda x: ' '.join(x))
    recipe_db['tags'] = recipe_db['tags'].str.title() 
    recipe_db['title'] = recipe_db['title'].map(lambda x: x[0])
    recipe_db['ingredient'] = recipe_db['ingredient'].map(lambda x: [x.strip() for x in x])
    recipe_db['ingredient'] = recipe_db['ingredient'].map(lambda x: ', '.join(x))
    if len(args) == 0:
        return recipe_db    
    elif len(args) > 1:
        proteins = list(args)
        recipe_db = recipe_db[~recipe_db['ingredient'].str.contains('|'.join(proteins))]
    else:
        recipe_db = recipe_db[~recipe_db['ingredient'].str.contains(args[0])]
    recipe_db = recipe_db.reset_index(drop=True)
    
    return recipe_db

In [86]:
# Dropping recipes that contain pork, beef or lamb.
recipe_db = drop_proteins(recipe_db, 'pork', 'beef', 'lamb')

In [87]:
# Recipe database.
recipe_db

,nutrition,title,instructions,servings,ingredient,tags,reviews,stars,link,image
0,"{'Calories': '387', 'Fat': '27.1 g (41.7%)', '...",How To Cook a Whole Chicken in the Slow Cooker,Create a base in the bottom of the slow cooker...,"4 to 6, depending on the size of the chicken","1 (3- to 4-pound) whole chicken, 1 teaspoon ko...",Alcohol Free Egg Free Paleo Dairy Free Low Car...,371,3.1,https://www.thekitchn.com/how-to-cook-a-whole-...,https://cdn.apartmenttherapy.info/image/fetch/...
1,"{'Calories': '914', 'Fat': '60.9 g (93.7%)', '...",Recipe: Italian Chicken Salad Sandwiches,Make the chicken salad: Arrange a rack in the ...,5 1/2 to 6 cups,"2 1/2 pounds bone-in, skin-on chicken breasts ...",Low Carb Autumn Chicken Dairy-Free Dairyfree E...,106,3.1,https://www.thekitchn.com/recipe-italian-chick...,https://cdn.apartmenttherapy.info/image/fetch/...
2,"{'Calories': '663', 'Fat': '43.3 g (66.6%)', '...",Recipe: Sheet Pan Chicken Souvlaki,Arrange a rack in the middle of the oven and h...,4,"2 pounds skin-on chicken thighs, bones removed...",Alcohol Free Egg Free Dairy Free Low Carb Fish...,346,2.7,https://www.thekitchn.com/recipe-sheet-pan-chi...,https://cdn.apartmenttherapy.info/image/fetch/...
3,"{'Calories': '402', 'Fat': '15.7 g (24.2%)', '...",Recipe: Parmesan Chicken and Kale Sauté,Heat the oil in a large skillet over medium he...,4,"2 tablespoons olive oil, 1 1/2 pounds boneless...",Egg Free Low Carb Fish Free Peanut Free Shellf...,600,2.9,https://www.thekitchn.com/recipe-parmesan-chic...,https://cdn.apartmenttherapy.info/image/fetch/...
4,"{'Calories': '334', 'Fat': '18.4 g (28.3%)', '...",Recipe: Slow Cooker Chicken Tikka Masala,"Place the chicken, yogurt, and 1 teaspoon of t...",4 to 6,"2 pounds boneless, skinless chicken thighs, cu...",Shellfish Free Low Carb Fish Free Alcohol Free...,1437,3.1,https://www.thekitchn.com/recipe-slow-cooker-c...,https://cdn.apartmenttherapy.info/image/fetch/...
5,"{'Calories': '516', 'Fat': '7.4 g (11.5%)', 'S...",Recipe: Gnocchi Skillet with Chicken Sausage &...,Bring a large pot of salted water to a boil ov...,4,"1 pound frozen or shelf-stable gnocchi, Kosher...",Shellfish Free Dairy Free Low Fat Fish Free Al...,483,3.1,https://www.thekitchn.com/recipe-gnocchi-skill...,https://cdn.apartmenttherapy.info/image/fetch/...
6,"{'Calories': '507', 'Fat': '20.2 g (31.0%)', '...",Recipe: Chicken Caesar Pizza,Arrange a rack in the middle of the oven and h...,4 to 6,"1 pound pizza dough, Olive oil, 6 tablespoons ...",Shellfish Free Fish Free Alcohol Free Peanut F...,1,5.0,https://www.thekitchn.com/recipe-chicken-caesa...,https://cdn.apartmenttherapy.info/image/fetch/...
7,"{'Calories': '605', 'Fat': '40.7 g (62.7%)', '...",Recipe: Sheet Pan Chicken with Red Potatoes an...,Arrange a rack in the middle of the oven and h...,4,"3 tablespoons olive oil, divided, 2 large bunc...",Alcohol Free Egg Free Dairy Free Low Carb Fish...,153,2.8,https://www.thekitchn.com/recipe-sheet-pan-chi...,https://cdn.apartmenttherapy.info/image/fetch/...
8,"{'Calories': '290', 'Fat': '18.5 g (28.5%)', '...",How To Make Classic Creamy Chicken Salad,Poach the chicken. Place the chicken and any a...,about 5 cups,"1 1/2 pounds boneless, skinless chicken breast...",Shellfish Free Low Carb Fish Free Alcohol Free...,46,2.9,https://www.thekitchn.com/hot-to-make-classic-...,https://cdn.apartmenttherapy.info/image/fetch/...
9,"{'Calories': '304', 'Fat': '6.4 g (9.8%)', 'Sa...",Recipe: Slow Cooker Honey Teriyaki Chicken,Arrange the chicken in a single layer in the b...,4 to 6,"2 pounds boneless, skinless chicken thighs (ab...",Shellfish Free Dairy Free Fish Free Alcohol Fr...,727,3.2,https://www.thekitchn.com/recipe-slow-cooker-h...,https://cdn.apartmenttherapy.info/image/fetch/...
